Import neccessary dependencies and api urls

In [26]:
import pandas as pd
import seaborn as sb
import requests as rq
import folium

from urls import SILVER_PRODUCTION_URL

In [27]:
def get_data(url: str = "", path: str = "") -> pd.DataFrame:

    entries_df = pd.DataFrame(
        columns = ["TIME_PERIOD", "OBS_VALUE", "COBER_GEO"]
    )

    # Data retrieval when a url is given
    if url != "":
        r = rq.get(url)
        data = r.json()['Series'][0]['OBSERVATIONS']

        for entry_no in range(len(data)):
            entries_list = [
                data[entry_no]["TIME_PERIOD"],
                data[entry_no]["OBS_VALUE"],
                data[entry_no]["COBER_GEO"],
            ]

            entries_df.loc[len(entries_df)] = entries_list

        return entries_df

    # Data retrieval when a file path is given
    elif path != "":
        # Extract the first 2 digits of the filename
        filename = path.split('/')[-1]
        cober_geo = filename.split('-')[0]
        
        data = pd.read_csv(
            path, 
            encoding='utf-16le',
            skiprows=4,
            skipfooter=3,
            engine='python'
        )
        
        data.columns = ['TIME_PERIOD', 'OBS_VALUE']

        data['OBS_VALUE'] = pd.to_numeric(data['OBS_VALUE'], errors='coerce').astype('Int64')
        
        data['COBER_GEO'] = cober_geo

        data = data.iloc[::-1]
        data = data.reset_index(drop=True)
        
        return data

In [28]:
with open("./filenames.txt", 'r') as file:
    data_df_list_silver = [get_data(path=f"./data/{filename.strip()}") for filename in file]

data_df_list_silver

[    TIME_PERIOD  OBS_VALUE COBER_GEO
 0       2025/08       4480        00
 1       2025/07       3956        00
 2       2025/06       4108        00
 3       2025/05       4263        00
 4       2025/04       3904        00
 ..          ...        ...       ...
 471     1986/05      50645        00
 472     1986/04      44418        00
 473     1986/03      52001        00
 474     1986/02      56816        00
 475     1986/01      34390        00
 
 [476 rows x 3 columns],
     TIME_PERIOD  OBS_VALUE COBER_GEO
 0       2025/08          0        02
 1       2025/07          0        02
 2       2025/06          0        02
 3       2025/05          0        02
 4       2025/04          0        02
 ..          ...        ...       ...
 363     1995/05       2544        02
 364     1995/04       2714        02
 365     1995/03       1139        02
 366     1995/02       1686        02
 367     1995/01        856        02
 
 [368 rows x 3 columns],
      TIME_PERIOD  OBS_VALUE COBER

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

JAWG_TOKEN = os.getenv("JAWG_TOKEN")

folium_map = folium.Map(
    location=[24, -102],
    zoom_start=5,
    tiles=f"https://tile.jawg.io/jawg-matrix/{{z}}/{{x}}/{{y}}.png?access-token={JAWG_TOKEN}",
    attr='Map tiles by <a href="https://www.jawg.io" target="_blank">Jawg</a>'
    )

folium_map